In [62]:
import soccerdata as sd
import pandas as pd
import time
import requests
import pandas as pd
import numpy as np

import warnings

from datetime import datetime, timedelta


warnings.filterwarnings('ignore')

In [63]:
df = pd.read_csv('EPL_data.csv')
df.head()

,week,day,date,time,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,season,game,home_starters,away_starters
0,1,Fri,2017-08-11,19:45,arsenal,2.5,4–3,1.5,leicester city,Mike Dean,e3c3ddf0,1848.286499,1716.994873,1718,2017-08-11 Arsenal-Leicester City,"['Petr Čech', 'Alexandre Lacazette', 'Mesut Öz...","['Kasper Schmeichel', 'Danny Simpson', 'Wes Mo..."
1,1,Sat,2017-08-12,17:30,brighton,0.3,0–2,1.9,manchester city,Michael Oliver,072bfc99,1583.799805,1866.807007,1718,2017-08-12 Brighton-Manchester City,"['Mathew Ryan', 'Markus Suttner', 'Bruno', 'Le...","['Ederson', 'Kyle Walker', 'Danilo', 'Vincent ..."
2,1,Sat,2017-08-12,15:00,chelsea,1.5,2–3,0.6,burnley,Craig Pawson,71b00bca,1909.399658,1628.988403,1718,2017-08-12 Chelsea-Burnley,"['Thibaut Courtois', 'Antonio Rüdiger', 'Marco...","['Tom Heaton', 'Matthew Lowton', 'Jack Cork', ..."
3,1,Sat,2017-08-12,15:00,crystal palace,1.1,0–3,1.5,huddersfield,Jonathan Moss,2d369d17,1642.862427,1475.799316,1718,2017-08-12 Crystal Palace-Huddersfield,"['Wayne Hennessey', 'Joel Ward', 'Patrick van ...","['Jonas Lössl', 'Tommy Smith', 'Philip Billing..."
4,1,Sat,2017-08-12,15:00,everton,0.6,1–0,0.4,stoke city,Niel Swarbrick,7c834541,1751.501343,1662.613770,1718,2017-08-12 Everton-Stoke City,"['Jordan Pickford', 'Morgan Schneiderlin', 'Le...","['Jack Butland', 'Erik Pieters', 'Joe Allen', ..."


In [64]:
df = df.drop(['day', 'time', 'game'], axis=1)

In [65]:
df.head()

,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,season,home_starters,away_starters
0,1,2017-08-11,arsenal,2.5,4–3,1.5,leicester city,Mike Dean,e3c3ddf0,1848.286499,1716.994873,1718,"['Petr Čech', 'Alexandre Lacazette', 'Mesut Öz...","['Kasper Schmeichel', 'Danny Simpson', 'Wes Mo..."
1,1,2017-08-12,brighton,0.3,0–2,1.9,manchester city,Michael Oliver,072bfc99,1583.799805,1866.807007,1718,"['Mathew Ryan', 'Markus Suttner', 'Bruno', 'Le...","['Ederson', 'Kyle Walker', 'Danilo', 'Vincent ..."
2,1,2017-08-12,chelsea,1.5,2–3,0.6,burnley,Craig Pawson,71b00bca,1909.399658,1628.988403,1718,"['Thibaut Courtois', 'Antonio Rüdiger', 'Marco...","['Tom Heaton', 'Matthew Lowton', 'Jack Cork', ..."
3,1,2017-08-12,crystal palace,1.1,0–3,1.5,huddersfield,Jonathan Moss,2d369d17,1642.862427,1475.799316,1718,"['Wayne Hennessey', 'Joel Ward', 'Patrick van ...","['Jonas Lössl', 'Tommy Smith', 'Philip Billing..."
4,1,2017-08-12,everton,0.6,1–0,0.4,stoke city,Niel Swarbrick,7c834541,1751.501343,1662.613770,1718,"['Jordan Pickford', 'Morgan Schneiderlin', 'Le...","['Jack Butland', 'Erik Pieters', 'Joe Allen', ..."


In [66]:


# Assuming df is your original dataframe
# Create a unified dataframe for both home and away xG values
games = df[['season', 'date', 'home_team', 'home_xg', 'away_team', 'away_xg']].copy()

# Create two dataframes: one for the home teams and one for the away teams
home_xg_df = games[['season', 'date', 'home_team', 'home_xg']].rename(columns={'home_team': 'team', 'home_xg': 'xg'})
away_xg_df = games[['season', 'date', 'away_team', 'away_xg']].rename(columns={'away_team': 'team', 'away_xg': 'xg'})

# Concatenate the home and away dataframes
all_xg_df = pd.concat([home_xg_df, away_xg_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_xg_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of xG for each team, grouped by season
all_xg_df['xg_cumulative'] = all_xg_df.groupby(['team', 'season'])['xg'].cumsum()

# Shift the cumsum by one, so we don't include the xG of the current game
all_xg_df['xg_cumulative'] = all_xg_df.groupby(['team', 'season'])['xg_cumulative'].shift(fill_value=0)

# Now we have the cumulative xG for each team up to but not including the current match
# Next, we merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_xg_df[['date', 'team', 'xg_cumulative']],
    left_on=['date', 'home_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'xg_cumulative': 'home_xG_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_xg_df[['date', 'team', 'xg_cumulative']],
    left_on=['date', 'away_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'xg_cumulative': 'away_xG_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Now your dataframe should have 'home_xG_to_date' and 'away_xG_to_date' columns with the correct cumulative xG values

In [67]:
df

,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,season,home_starters,away_starters,home_xG_to_date,away_xG_to_date
0,1,2017-08-11,arsenal,2.5,4–3,1.5,leicester city,Mike Dean,e3c3ddf0,1848.286499,1716.994873,1718,"['Petr Čech', 'Alexandre Lacazette', 'Mesut Öz...","['Kasper Schmeichel', 'Danny Simpson', 'Wes Mo...",0.0,0.0
1,1,2017-08-12,brighton,0.3,0–2,1.9,manchester city,Michael Oliver,072bfc99,1583.799805,1866.807007,1718,"['Mathew Ryan', 'Markus Suttner', 'Bruno', 'Le...","['Ederson', 'Kyle Walker', 'Danilo', 'Vincent ...",0.0,0.0
2,1,2017-08-12,chelsea,1.5,2–3,0.6,burnley,Craig Pawson,71b00bca,1909.399658,1628.988403,1718,"['Thibaut Courtois', 'Antonio Rüdiger', 'Marco...","['Tom Heaton', 'Matthew Lowton', 'Jack Cork', ...",0.0,0.0
3,1,2017-08-12,crystal palace,1.1,0–3,1.5,huddersfield,Jonathan Moss,2d369d17,1642.862427,1475.799316,1718,"['Wayne Hennessey', 'Joel Ward', 'Patrick van ...","['Jonas Lössl', 'Tommy Smith', 'Philip Billing...",0.0,0.0
4,1,2017-08-12,everton,0.6,1–0,0.4,stoke city,Niel Swarbrick,7c834541,1751.501343,1662.613770,1718,"['Jordan Pickford', 'Morgan Schneiderlin', 'Le...","['Jack Butland', 'Erik Pieters', 'Joe Allen', ...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,38,2023-05-28,everton,1.0,1–0,0.5,bournemouth,Stuart Attwell,94de848f,1698.783569,1662.989624,2223,"['Jordan Pickford', 'James Tarkowski', 'Dwight...","['Mark Travers', 'Lloyd Kelly', 'David Brooks'...",44.3,37.8
2276,38,2023-05-28,leeds united,1.5,1–4,2.2,tottenham,Anthony Taylor,c9c73ddd,1664.983643,1802.854126,2223,"['Joel Robles', 'Luke Ayling', 'Adam Forshaw',...","['Fraser Forster', 'Oliver Skipp', 'Davinson S...",46.0,55.1
2277,38,2023-05-28,leicester city,1.4,2–1,1.4,west ham,Simon Hooper,a96c9915,1713.210815,1776.454712,2223,"['Daniel Iversen', 'Wout Faes', 'Jonny Evans',...","['Łukasz Fabiański', 'Aaron Cresswell', 'Vladi...",49.1,47.8
2278,38,2023-05-28,manchester utd,2.9,2–1,1.8,fulham,Robert Jones,64823dc9,1866.215576,1736.953369,2223,"['David de Gea', 'Victor Lindelöf', 'Harry Mag...","['Bernd Leno', 'Kenny Tete', 'Tosin Adarabioyo...",64.6,44.5


In [68]:
#Checking if End of season xG values are correct


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 38]

arsenal_week38


,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,season,home_starters,away_starters,home_xG_to_date,away_xG_to_date
372,38,2018-05-13,huddersfield,1.7,0–1,2.0,arsenal,Michael Oliver,7c431214,1571.973755,1821.480469,1718,"['Jonas Lössl', 'Terence Kongolo', 'Jonathan H...","['David Ospina', 'Henrikh Mkhitaryan', 'Aaron ...",29.7,66.4
751,38,2019-05-12,burnley,1.7,1–3,2.7,arsenal,Mike Dean,302b231e,1692.925781,1877.940063,1819,"['Tom Heaton', 'Matthew Lowton', 'Charlie Tayl...","['Bernd Leno', 'Mohamed Elneny', 'Henrikh Mkhi...",42.9,57.4
1130,38,2020-07-26,arsenal,2.3,3–2,3.7,watford,Mike Dean,5ecaeb4b,1804.914307,1658.365479,1920,"['Emiliano Martínez', 'Kieran Tierney', 'Dani ...","['Ben Foster', 'Craig Dawson', 'Troy Deeney', ...",44.7,45.3
1510,38,2021-05-23,arsenal,0.8,2–0,0.2,brighton,Jonathan Moss,9f6c52f8,1861.241943,1728.190918,2021,"['Bernd Leno', 'Kieran Tierney', 'Gabriel Dos ...","['Robert Sánchez', 'Ben White', 'Adam Webster'...",51.0,50.8
1890,38,2022-05-22,arsenal,4.2,5–1,1.1,everton,Andre Marriner,4100d195,1846.874878,1709.672852,2122,"['Aaron Ramsdale', 'Gabriel Dos Santos', 'Buka...","['Asmir Begović', 'Jonjoe Kenny', 'Mason Holga...",56.2,40.3
2270,38,2023-05-28,arsenal,2.8,5–0,0.5,wolves,Andre Marriner,71d5bd41,1909.242920,1722.064697,2223,"['Aaron Ramsdale', 'Ben White', 'Thomas Partey...","['José Sá', 'Nathan Collins', 'Mario Lemina', ...",68.9,36.6


In [69]:
# Assuming df is your original dataframe with the structure mentioned before
# Create two dataframes: one for the home team xG against and one for the away team xG against
home_xg_against_df = games[['season', 'date', 'home_team', 'away_xg']].rename(columns={'home_team': 'team', 'away_xg': 'xg_against'})
away_xg_against_df = games[['season', 'date', 'away_team', 'home_xg']].rename(columns={'away_team': 'team', 'home_xg': 'xg_against'})

# Concatenate the home and away xG against dataframes
all_xg_against_df = pd.concat([home_xg_against_df, away_xg_against_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_xg_against_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of xG against for each team, grouped by season
all_xg_against_df['xg_against_cumulative'] = all_xg_against_df.groupby(['team', 'season'])['xg_against'].cumsum()

# Shift the cumsum by one, so we don't include the xG against of the current game
all_xg_against_df['xg_against_cumulative'] = all_xg_against_df.groupby(['team', 'season'])['xg_against_cumulative'].shift(fill_value=0)

# Now we have the cumulative xG against for each team up to but not including the current match
# Next, we merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_xg_against_df[['date', 'team', 'xg_against_cumulative']],
    left_on=['date', 'home_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'xg_against_cumulative': 'home_xG_against_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_xg_against_df[['date', 'team', 'xg_against_cumulative']],
    left_on=['date', 'away_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'xg_against_cumulative': 'away_xG_against_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Now your dataframe should have 'home_xG_against_to_date' and 'away_xG_against_to_date' columns with the correct cumulative xG against values


In [70]:
df.head()

,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,season,home_starters,away_starters,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date
0,1,2017-08-11,arsenal,2.5,4–3,1.5,leicester city,Mike Dean,e3c3ddf0,1848.286499,1716.994873,1718,"['Petr Čech', 'Alexandre Lacazette', 'Mesut Öz...","['Kasper Schmeichel', 'Danny Simpson', 'Wes Mo...",0.0,0.0,0.0,0.0
1,1,2017-08-12,brighton,0.3,0–2,1.9,manchester city,Michael Oliver,072bfc99,1583.799805,1866.807007,1718,"['Mathew Ryan', 'Markus Suttner', 'Bruno', 'Le...","['Ederson', 'Kyle Walker', 'Danilo', 'Vincent ...",0.0,0.0,0.0,0.0
2,1,2017-08-12,chelsea,1.5,2–3,0.6,burnley,Craig Pawson,71b00bca,1909.399658,1628.988403,1718,"['Thibaut Courtois', 'Antonio Rüdiger', 'Marco...","['Tom Heaton', 'Matthew Lowton', 'Jack Cork', ...",0.0,0.0,0.0,0.0
3,1,2017-08-12,crystal palace,1.1,0–3,1.5,huddersfield,Jonathan Moss,2d369d17,1642.862427,1475.799316,1718,"['Wayne Hennessey', 'Joel Ward', 'Patrick van ...","['Jonas Lössl', 'Tommy Smith', 'Philip Billing...",0.0,0.0,0.0,0.0
4,1,2017-08-12,everton,0.6,1–0,0.4,stoke city,Niel Swarbrick,7c834541,1751.501343,1662.613770,1718,"['Jordan Pickford', 'Morgan Schneiderlin', 'Le...","['Jack Butland', 'Erik Pieters', 'Joe Allen', ...",0.0,0.0,0.0,0.0


In [71]:
# Replace en dash '–' with standard hyphen '-' before splitting
df['score'] = df['score'].str.replace('–', '-', regex=False)

# Split the 'score' column into two new columns 'home_goals_scored' and 'away_goals_scored'
df[['home_goals_scored', 'away_goals_scored']] = df['score'].str.split('-', expand=True).astype(int)

# Now df has two new columns with the home and away goals as integers


In [72]:
df

,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,away_team_elo,season,home_starters,away_starters,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date,home_goals_scored,away_goals_scored
0,1,2017-08-11,arsenal,2.5,4-3,1.5,leicester city,Mike Dean,e3c3ddf0,1848.286499,1716.994873,1718,"['Petr Čech', 'Alexandre Lacazette', 'Mesut Öz...","['Kasper Schmeichel', 'Danny Simpson', 'Wes Mo...",0.0,0.0,0.0,0.0,4,3
1,1,2017-08-12,brighton,0.3,0-2,1.9,manchester city,Michael Oliver,072bfc99,1583.799805,1866.807007,1718,"['Mathew Ryan', 'Markus Suttner', 'Bruno', 'Le...","['Ederson', 'Kyle Walker', 'Danilo', 'Vincent ...",0.0,0.0,0.0,0.0,0,2
2,1,2017-08-12,chelsea,1.5,2-3,0.6,burnley,Craig Pawson,71b00bca,1909.399658,1628.988403,1718,"['Thibaut Courtois', 'Antonio Rüdiger', 'Marco...","['Tom Heaton', 'Matthew Lowton', 'Jack Cork', ...",0.0,0.0,0.0,0.0,2,3
3,1,2017-08-12,crystal palace,1.1,0-3,1.5,huddersfield,Jonathan Moss,2d369d17,1642.862427,1475.799316,1718,"['Wayne Hennessey', 'Joel Ward', 'Patrick van ...","['Jonas Lössl', 'Tommy Smith', 'Philip Billing...",0.0,0.0,0.0,0.0,0,3
4,1,2017-08-12,everton,0.6,1-0,0.4,stoke city,Niel Swarbrick,7c834541,1751.501343,1662.613770,1718,"['Jordan Pickford', 'Morgan Schneiderlin', 'Le...","['Jack Butland', 'Erik Pieters', 'Joe Allen', ...",0.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,38,2023-05-28,everton,1.0,1-0,0.5,bournemouth,Stuart Attwell,94de848f,1698.783569,1662.989624,2223,"['Jordan Pickford', 'James Tarkowski', 'Dwight...","['Mark Travers', 'Lloyd Kelly', 'David Brooks'...",44.3,37.8,65.2,62.7,1,0
2276,38,2023-05-28,leeds united,1.5,1-4,2.2,tottenham,Anthony Taylor,c9c73ddd,1664.983643,1802.854126,2223,"['Joel Robles', 'Luke Ayling', 'Adam Forshaw',...","['Fraser Forster', 'Oliver Skipp', 'Davinson S...",46.0,55.1,64.9,48.0,1,4
2277,38,2023-05-28,leicester city,1.4,2-1,1.4,west ham,Simon Hooper,a96c9915,1713.210815,1776.454712,2223,"['Daniel Iversen', 'Wout Faes', 'Jonny Evans',...","['Łukasz Fabiański', 'Aaron Cresswell', 'Vladi...",49.1,47.8,62.0,51.6,2,1
2278,38,2023-05-28,manchester utd,2.9,2-1,1.8,fulham,Robert Jones,64823dc9,1866.215576,1736.953369,2223,"['David de Gea', 'Victor Lindelöf', 'Harry Mag...","['Bernd Leno', 'Kenny Tete', 'Tosin Adarabioyo...",64.6,44.5,48.5,60.8,2,1


In [73]:
# Assuming df is your original dataframe and you already have 'home_goals_scored' and 'away_goals_scored' columns
# Create a unified dataframe for both home and away goals scored values
games_goals = df[['season', 'date', 'home_team', 'home_goals_scored', 'away_team', 'away_goals_scored']].copy()

# Create two dataframes: one for the home teams' goals and one for the away teams' goals
home_goals_df = games_goals[['season', 'date', 'home_team', 'home_goals_scored']].rename(columns={'home_team': 'team', 'home_goals_scored': 'goals'})
away_goals_df = games_goals[['season', 'date', 'away_team', 'away_goals_scored']].rename(columns={'away_team': 'team', 'away_goals_scored': 'goals'})

# Concatenate the home and away dataframes
all_goals_df = pd.concat([home_goals_df, away_goals_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_goals_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of goals for each team, grouped by season
all_goals_df['goals_cumulative'] = all_goals_df.groupby(['team', 'season'])['goals'].cumsum()

# Shift the cumsum by one, so we don't include the goals of the current game
all_goals_df['goals_cumulative'] = all_goals_df.groupby(['team', 'season'])['goals_cumulative'].shift(fill_value=0)

# Now we have the cumulative goals for each team up to but not including the current match
# Next, we merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_goals_df[['date', 'team', 'goals_cumulative']],
    left_on=['date', 'home_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'goals_cumulative': 'home_goals_scored_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_goals_df[['date', 'team', 'goals_cumulative']],
    left_on=['date', 'away_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'goals_cumulative': 'away_goals_scored_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Now your dataframe should have 'home_goals_scored_to_date' and 'away_goals_scored_to_date' columns with the correct cumulative goals values


In [74]:
#Sanity Checker!!


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 38]

arsenal_week38


,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,...,home_starters,away_starters,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date,home_goals_scored,away_goals_scored,home_goals_scored_to_date,away_goals_scored_to_date
372,38,2018-05-13,huddersfield,1.7,0-1,2.0,arsenal,Michael Oliver,7c431214,1571.973755,...,"['Jonas Lössl', 'Terence Kongolo', 'Jonathan H...","['David Ospina', 'Henrikh Mkhitaryan', 'Aaron ...",29.7,66.4,46.6,46.1,0,1,28,73
751,38,2019-05-12,burnley,1.7,1-3,2.7,arsenal,Mike Dean,302b231e,1692.925781,...,"['Tom Heaton', 'Matthew Lowton', 'Charlie Tayl...","['Bernd Leno', 'Mohamed Elneny', 'Henrikh Mkhi...",42.9,57.4,59.3,52.5,1,3,44,70
1130,38,2020-07-26,arsenal,2.3,3-2,3.7,watford,Mike Dean,5ecaeb4b,1804.914307,...,"['Emiliano Martínez', 'Kieran Tierney', 'Dani ...","['Ben Foster', 'Craig Dawson', 'Troy Deeney', ...",44.7,45.3,51.9,55.8,3,2,53,34
1510,38,2021-05-23,arsenal,0.8,2-0,0.2,brighton,Jonathan Moss,9f6c52f8,1861.241943,...,"['Bernd Leno', 'Kieran Tierney', 'Gabriel Dos ...","['Robert Sánchez', 'Ben White', 'Adam Webster'...",51.0,50.8,42.8,34.6,2,0,53,40
1890,38,2022-05-22,arsenal,4.2,5-1,1.1,everton,Andre Marriner,4100d195,1846.874878,...,"['Aaron Ramsdale', 'Gabriel Dos Santos', 'Buka...","['Asmir Begović', 'Jonjoe Kenny', 'Mason Holga...",56.2,40.3,44.5,51.0,5,1,56,42
2270,38,2023-05-28,arsenal,2.8,5-0,0.5,wolves,Andre Marriner,71d5bd41,1909.242920,...,"['Aaron Ramsdale', 'Ben White', 'Thomas Partey...","['José Sá', 'Nathan Collins', 'Mario Lemina', ...",68.9,36.6,41.7,57.6,5,0,83,31


In [75]:
# Assuming df is your original dataframe and you already have 'home_goals_scored' and 'away_goals_scored' columns
# Create a unified dataframe for both home and away goals conceded values
games_goals = df[['season', 'date', 'home_team', 'away_goals_scored', 'away_team', 'home_goals_scored']].copy()

# For goals conceded, we take the goals scored by the opposite team
# Create two dataframes: one for the home teams' goals conceded and one for the away teams' goals conceded
home_goals_conceded_df = games_goals[['season', 'date', 'home_team', 'away_goals_scored']].rename(columns={'home_team': 'team', 'away_goals_scored': 'goals_conceded'})
away_goals_conceded_df = games_goals[['season', 'date', 'away_team', 'home_goals_scored']].rename(columns={'away_team': 'team', 'home_goals_scored': 'goals_conceded'})

# Concatenate the home and away dataframes
all_goals_conceded_df = pd.concat([home_goals_conceded_df, away_goals_conceded_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_goals_conceded_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of goals conceded for each team, grouped by season
all_goals_conceded_df['goals_conceded_cumulative'] = all_goals_conceded_df.groupby(['team', 'season'])['goals_conceded'].cumsum()

# Shift the cumsum by one, so we don't include the goals conceded of the current game
all_goals_conceded_df['goals_conceded_cumulative'] = all_goals_conceded_df.groupby(['team', 'season'])['goals_conceded_cumulative'].shift(fill_value=0)

# Now we have the cumulative goals conceded for each team up to but not including the current match
# Next, we merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_goals_conceded_df[['date', 'team', 'goals_conceded_cumulative']],
    left_on=['date', 'home_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'goals_conceded_cumulative': 'home_goals_conceded_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_goals_conceded_df[['date', 'team', 'goals_conceded_cumulative']],
    left_on=['date', 'away_team'],
    right_on=['date', 'team'],
    how='left'
)
df.rename(columns={'goals_conceded_cumulative': 'away_goals_conceded_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Now your dataframe should have 'home_goals_conceded_to_date' and 'away_goals_conceded_to_date' columns with the correct cumulative goals conceded values


In [76]:
#Sanity Checker!!


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 38]

arsenal_week38


,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,...,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date,home_goals_scored,away_goals_scored,home_goals_scored_to_date,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date
372,38,2018-05-13,huddersfield,1.7,0-1,2.0,arsenal,Michael Oliver,7c431214,1571.973755,...,29.7,66.4,46.6,46.1,0,1,28,73,57,51
751,38,2019-05-12,burnley,1.7,1-3,2.7,arsenal,Mike Dean,302b231e,1692.925781,...,42.9,57.4,59.3,52.5,1,3,44,70,65,50
1130,38,2020-07-26,arsenal,2.3,3-2,3.7,watford,Mike Dean,5ecaeb4b,1804.914307,...,44.7,45.3,51.9,55.8,3,2,53,34,46,61
1510,38,2021-05-23,arsenal,0.8,2-0,0.2,brighton,Jonathan Moss,9f6c52f8,1861.241943,...,51.0,50.8,42.8,34.6,2,0,53,40,39,44
1890,38,2022-05-22,arsenal,4.2,5-1,1.1,everton,Andre Marriner,4100d195,1846.874878,...,56.2,40.3,44.5,51.0,5,1,56,42,47,61
2270,38,2023-05-28,arsenal,2.8,5-0,0.5,wolves,Andre Marriner,71d5bd41,1909.242920,...,68.9,36.6,41.7,57.6,5,0,83,31,43,53


In [77]:
df.columns

Index(['week', 'date', 'home_team', 'home_xg', 'score', 'away_xg', 'away_team',
       'referee', 'game_id', 'home_team_elo', 'away_team_elo', 'season',
       'home_starters', 'away_starters', 'home_xG_to_date', 'away_xG_to_date',
       'home_xG_against_to_date', 'away_xG_against_to_date',
       'home_goals_scored', 'away_goals_scored', 'home_goals_scored_to_date',
       'away_goals_scored_to_date', 'home_goals_conceded_to_date',
       'away_goals_conceded_to_date'],
      dtype='object')

In [78]:
# Apply the function to each row and create new columns for home and away match points
df[['home_match_points', 'away_match_points']] = df.apply(
    lambda row: calculate_points(row['home_goals_scored'], row['away_goals_scored']),
    axis=1, result_type='expand'
)

# Create a unified dataframe for both home and away points values
points_df = df[['season', 'date', 'home_team', 'home_match_points', 'away_team', 'away_match_points']].copy()

# Create two dataframes: one for the home teams and one for the away teams
home_points_df = points_df[['season', 'date', 'home_team', 'home_match_points']].rename(columns={'home_team': 'team', 'home_match_points': 'points'})
away_points_df = points_df[['season', 'date', 'away_team', 'away_match_points']].rename(columns={'away_team': 'team', 'away_match_points': 'points'})

# Concatenate the home and away dataframes
all_points_df = pd.concat([home_points_df, away_points_df], ignore_index=True)

# Sort the dataframe by team and date to ensure correct cumsum
all_points_df.sort_values(by=['team', 'season', 'date'], inplace=True)

# Calculate the cumulative sum of points for each team, grouped by season
all_points_df['points_cumulative'] = all_points_df.groupby(['team', 'season'])['points'].cumsum()

# Shift the cumsum by one, so we don't include the points of the current game
all_points_df['points_cumulative'] = all_points_df.groupby(['team', 'season'])['points_cumulative'].shift(fill_value=0)

# Merge this back into the original dataframe
# We need to merge twice, once for the home team and once for the away team

# Merge for home_team
df = df.merge(
    all_points_df[['season', 'date', 'team', 'points_cumulative']],
    left_on=['season', 'date', 'home_team'],
    right_on=['season', 'date', 'team'],
    how='left'
)
df.rename(columns={'points_cumulative': 'home_points_to_date'}, inplace=True)

# Merge for away_team
df = df.merge(
    all_points_df[['season', 'date', 'team', 'points_cumulative']],
    left_on=['season', 'date', 'away_team'],
    right_on=['season', 'date', 'team'],
    how='left'
)
df.rename(columns={'points_cumulative': 'away_points_to_date'}, inplace=True)

# Drop the extra 'team' columns added during the merge
df.drop(columns=['team_x', 'team_y'], inplace=True)

# Your dataframe now has 'home_points_to_date' and 'away_points_to_date' columns with the correct cumulative points values, reset for each season and not including the current game.


In [79]:
#Sanity Checker!!


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 38]

arsenal_week38


,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,...,home_goals_scored,away_goals_scored,home_goals_scored_to_date,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date,home_match_points,away_match_points,home_points_to_date,away_points_to_date
372,38,2018-05-13,huddersfield,1.7,0-1,2.0,arsenal,Michael Oliver,7c431214,1571.973755,...,0,1,28,73,57,51,0,3,37,60
751,38,2019-05-12,burnley,1.7,1-3,2.7,arsenal,Mike Dean,302b231e,1692.925781,...,1,3,44,70,65,50,0,3,40,67
1130,38,2020-07-26,arsenal,2.3,3-2,3.7,watford,Mike Dean,5ecaeb4b,1804.914307,...,3,2,53,34,46,61,3,0,53,34
1510,38,2021-05-23,arsenal,0.8,2-0,0.2,brighton,Jonathan Moss,9f6c52f8,1861.241943,...,2,0,53,40,39,44,3,0,58,41
1890,38,2022-05-22,arsenal,4.2,5-1,1.1,everton,Andre Marriner,4100d195,1846.874878,...,5,1,56,42,47,61,3,0,66,39
2270,38,2023-05-28,arsenal,2.8,5-0,0.5,wolves,Andre Marriner,71d5bd41,1909.242920,...,5,0,83,31,43,53,3,0,81,41


In [80]:
df.columns

Index(['week', 'date', 'home_team', 'home_xg', 'score', 'away_xg', 'away_team',
       'referee', 'game_id', 'home_team_elo', 'away_team_elo', 'season',
       'home_starters', 'away_starters', 'home_xG_to_date', 'away_xG_to_date',
       'home_xG_against_to_date', 'away_xG_against_to_date',
       'home_goals_scored', 'away_goals_scored', 'home_goals_scored_to_date',
       'away_goals_scored_to_date', 'home_goals_conceded_to_date',
       'away_goals_conceded_to_date', 'home_match_points', 'away_match_points',
       'home_points_to_date', 'away_points_to_date'],
      dtype='object')

In [93]:

# Function to calculate form for the last 5 games for a given team and season
def calculate_form(team, season, df):
    # Filter matches for the current team and season
    team_matches = df[(df['season'] == season) & ((df['home_team'] == team) | (df['away_team'] == team))]
    
    # Determine match points based on whether the team was home or away
    team_matches['match_points'] = team_matches.apply(
        lambda x: x['home_match_points'] if x['home_team'] == team else x['away_match_points'], axis=1
    )
    
    # Calculate the rolling sum of the last 5 games' points, not including the current match
    rolling_points = team_matches['match_points'].rolling(window=5, min_periods=1).sum().shift()
    
    # Get the last value of the rolling sum which represents the form of the last 5 matches
    last_value = rolling_points.iloc[-1] if not rolling_points.empty else 0
    
    # Return the form
    return last_value / 5

# Assuming 'df' is your DataFrame with match data
# Initialize the columns with zeros
df['home_form'] = 0
df['away_form'] = 0

# Loop over the dataframe and calculate form for each match
for index, row in df.iterrows():
    # Calculate home and away form if week is greater than 5, otherwise set to 0
    if row['week'] > 5:
        df.at[index, 'home_form'] = calculate_form(row['home_team'], row['season'], df)
        df.at[index, 'away_form'] = calculate_form(row['away_team'], row['season'], df)
    else:
        df.at[index, 'home_form'] = 0
        df.at[index, 'away_form'] = 0





      week       home_team        away_team  season  home_form  away_form
0        1         arsenal   leicester city    1718        0.0        0.0
1        1        brighton  manchester city    1718        0.0        0.0
2        1         chelsea          burnley    1718        0.0        0.0
3        1  crystal palace     huddersfield    1718        0.0        0.0
4        1         everton       stoke city    1718        0.0        0.0
...    ...             ...              ...     ...        ...        ...
2275    38         everton      bournemouth    2223        1.0        1.2
2276    38    leeds united        tottenham    2223        0.4        0.8
2277    38  leicester city         west ham    2223        0.6        1.2
2278    38  manchester utd           fulham    2223        1.8        1.4
2279    38     southampton        liverpool    2223        0.0        2.6

[2280 rows x 6 columns]


In [96]:
# You can now check your DataFrame for the 'home_form' and 'away_form' columns
df[['week', 'home_team', 'away_team', 'season', 'home_form', 'away_form']].head(20)


,week,home_team,away_team,season,home_form,away_form
0,1,arsenal,leicester city,1718,0.0,0.0
1,1,brighton,manchester city,1718,0.0,0.0
2,1,chelsea,burnley,1718,0.0,0.0
3,1,crystal palace,huddersfield,1718,0.0,0.0
4,1,everton,stoke city,1718,0.0,0.0
5,1,southampton,swansea city,1718,0.0,0.0
6,1,watford,liverpool,1718,0.0,0.0
7,1,west brom,bournemouth,1718,0.0,0.0
8,1,manchester utd,west ham,1718,0.0,0.0
9,1,newcastle utd,tottenham,1718,0.0,0.0


In [97]:
#Sanity Checker!!


# Filter for all Arsenal games
arsenal_games = df[(df['home_team'] == 'arsenal') | (df['away_team'] == 'arsenal')]

# Sort the games by date to see the progression of the 'home_xG_to_date' and 'away_xG_to_date' columns
arsenal_sorted = arsenal_games.sort_values(by='date')

# Display the dataframe for week 38
arsenal_week38 = arsenal_sorted[arsenal_sorted['week'] == 38]

arsenal_week38


,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,...,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date,home_match_points,away_match_points,home_points_to_date,away_points_to_date,match_points,home_form,away_form
372,38,2018-05-13,huddersfield,1.7,0-1,2.0,arsenal,Michael Oliver,7c431214,1571.973755,...,73,57,51,0,3,37,60,3,1.2,1.2
751,38,2019-05-12,burnley,1.7,1-3,2.7,arsenal,Mike Dean,302b231e,1692.925781,...,70,65,50,0,3,40,67,3,1.4,0.8
1130,38,2020-07-26,arsenal,2.3,3-2,3.7,watford,Mike Dean,5ecaeb4b,1804.914307,...,34,46,61,3,0,53,34,3,1.4,1.2
1510,38,2021-05-23,arsenal,0.8,2-0,0.2,brighton,Jonathan Moss,9f6c52f8,1861.241943,...,40,39,44,3,0,58,41,3,2.4,1.4
1890,38,2022-05-22,arsenal,4.2,5-1,1.1,everton,Andre Marriner,4100d195,1846.874878,...,42,47,61,3,0,66,39,3,1.8,2.0
2270,38,2023-05-28,arsenal,2.8,5-0,0.5,wolves,Andre Marriner,71d5bd41,1909.242920,...,31,43,53,3,0,81,41,3,1.2,1.4


In [99]:
# Conditions for the different classes based on 'home_match_points'
conditions = [
    df['home_match_points'] == 3,
    df['home_match_points'] == 1,
    df['home_match_points'] == 0,
]

# Corresponding class labels for the target column
class_labels = [0, 1, 2]

# Create the target column 'match_result'
df['match_result'] = np.select(conditions, class_labels)

# Now your DataFrame will have a 'match_result' column with values 0, 1, or 2 based on 'home_match_points'
print(df[['home_match_points', 'match_result']])


      home_match_points  match_result
0                     3             0
1                     0             2
2                     0             2
3                     0             2
4                     3             0
...                 ...           ...
2275                  3             0
2276                  0             2
2277                  3             0
2278                  3             0
2279                  1             1

[2280 rows x 2 columns]


In [100]:
csv_file_path = '/Users/lkimball/Desktop/Flatiron/CapstoneProject/EPL_Data_featured.csv'
df.to_csv(csv_file_path, index=False)